
### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2020 Semester 1

## Assignment 1: Naive Bayes Classifiers

###### Submission deadline: 7 pm, Monday 20 Apr 2020

**Student Name(s):**    `PLEASE ENTER YOUR NAME(S) HERE`

**Student ID(s):**     `PLEASE ENTER YOUR ID(S) HERE`


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [168]:
# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing
import pandas as pd
from collections import defaultdict
import numpy as np

def preprocess(filename, classcolumn,id_code):
    temp = pd.read_csv(filename, skiprows=0, header=None, delimiter=';', skip_blank_lines=True)
    df=temp[0].str.split(',',expand=True)
    class1= df[classcolumn-1]
    df.drop(labels=[classcolumn-1], axis=1,inplace = True)
    df.insert(len(df.columns), 'class', class1)
    if id_code != 'none':
        del df[id_code]
    data=df.values.tolist()
    
    return data


In [169]:
def prior_func(data):
    prob={}
    for row in data: 
        if row[-1] in prob:
            prob[row[-1]]+=1
        else:
            prob[row[-1]]=1
    for row in prob.keys():
        prob[row]=prob[row]/len(data)
    return prob


In [170]:
def posterior_func(data):
    prob = defaultdict(lambda : defaultdict(lambda : defaultdict(int)))
    
    for row in data:
        attr= row[:-1]
        classes= row[-1]
        for attr, a in enumerate(attr):
            prob[classes][attr][a] += 1
    for classes in prob:
        for attr in prob[classes]:
            l = sum(prob[classes][attr].values())
            for a in prob[classes][attr]:
                 prob[classes][attr][a] =  prob[classes][attr][a]/l
    
    
    prob=dict(prob)
    for classes in prob:
        prob[classes]=dict(prob[classes])
        for attr in prob[classes]:
             prob[classes][attr]=dict( prob[classes][attr])
    return prob

In [171]:
# This function should calculat prior probabilities and likelihoods from the training data and using
# them to build a naive Bayes model

def train(data):
    return (prior_func(data),posterior_func(data))


In [172]:
# This function should predict classes for new items in a test dataset (for the purposes of this assignment, you
# can re-use the training data as a test set)
def predict(preprocessed_data, trained_set):
   
    prior_prob, post_prob= trained_set[0], trained_set[1]
    length= len(preprocessed_data)
    predicted_class=[]
    for row in preprocessed_data:
        prediction ={}
        for a, b in prior_prob.items():
            prediction[a]=b
            for i in range(len(row)-1):
                if row[i] in post_prob[a][i]:
                    prediction[a] =prediction[a]* post_prob[a][i][row[i]]
                else:
                    prediction[a]= prediction[a]*0.01
                    prediction[a]= prediction[a]/length
        max_key = max(prediction, key=prediction.get)
        predicted_class.append(max_key)
    
    return predicted_class


In [173]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels

def evaluate(filename, classcolumn,id_code):
    preprocessed_data=preprocess(filename, classcolumn,id_code)
    trained_set=train(preprocessed_data)
    total=0
    predict1=predict(preprocessed_data, trained_set)
    count=len(preprocessed_data)
    
    for a in range(count):
        if  preprocessed_data[a][-1]== predict1[a]:
            total=total+1
    
    
    accuracy= total/count
    
    
    return accuracy*100


In [184]:
list_datasets=[["breast-cancer-wisconsin.data",11,1],["mushroom.data",1,"none"],["lymphography.data",1,"none"],
               ["wine.data",1,"none"],["car.data",7,"none"],["nursery.data",9,"none"],["somerville.data",1,"none"],
               ["adult.data",15,"none"],["bank.data",15,"none"],["university.data",14,0],["wdbc.data",2,0]]

for a in range(len(list_datasets)):
    print(str(list_datasets[a][0])+'      '+str(evaluate(list_datasets[a][0],list_datasets[a][1],list_datasets[a][2])))
    

breast-cancer-wisconsin.data      98.14020028612303
mushroom.data      99.1506646971935
lymphography.data      89.1891891891892
wine.data      100.0
car.data      87.38425925925925
nursery.data      90.30864197530865
somerville.data      67.13286713286713
adult.data      93.78704585240011
bank.data      92.33814779589038
university.data      93.10344827586206
wdbc.data      100.0


## Questions 


If you are in a group of 1, you will respond to question (1), and **one** other of your choosing (two responses in total).

If you are in a group of 2, you will respond to question (1) and question (2), and **two** others of your choosing (four responses in total). 

A response to a question should take about 100–250 words, and make reference to the data wherever possible.

#### NOTE: you may develope codes or functions in respond to the question, but your formal answer should be added to a separate file.

### Q1
Try discretising the numeric attributes in these datasets and treating them as discrete variables in the na¨ıve Bayes classifier. You can use a discretisation method of your choice and group the numeric values into any number of levels (but around 3 to 5 levels would probably be a good starting point). Does discretizing the variables improve classification performance, compared to the Gaussian na¨ıve Bayes approach? Why or why not?

In [296]:
from pandas import DataFrame
import math
import pandas
df=preprocess("wdbc.data",2,0)
df= DataFrame(df)

classes= df[30]
for i in range(30):
    k=0
    k=df[i].astype(float).max()
    for j in range(569):
        df[i][j]= math.floor(math.ceil(float(df[i][j]) * 5) / k)
print (df)
df.to_csv(r'C:\Users\ANEESH CHATTARAJ\Desktop\comp30027\2020\assignment1_2020_datasets\Name11.data', index = False)

evaluate('Name11.data',30,"none")




    0  1  2  3  4  5  6  7  8   9  ... 21 22 23 24 25 26 27 28 29 30
0    3  1  3  2  6  5  4  4  6  10 ...  1  3  2  4  3  3  6  4  4  M
1    3  2  3  2  6  2  2  4  3  10 ...  2  3  2  4  0  1  3  3  4  M
2    3  2  3  2  6  2  2  4  6  10 ...  2  3  2  4  2  2  6  3  4  M
3    2  2  2  0  6  5  4  4  6  10 ...  2  1  0  8  4  3  6  6  4  M
4    3  1  3  2  6  2  2  4  3  10 ...  1  3  1  4  1  1  3  3  4  M
5    2  2  2  0  6  2  2  4  6  10 ...  2  2  0  4  2  2  3  3  4  M
6    3  2  3  2  6  2  2  4  3  10 ...  2  3  1  4  1  1  3  3  4  M
7    2  2  2  1  6  2  2  4  6  10 ...  2  2  1  4  1  1  3  3  4  M
8    2  2  2  1  6  2  2  4  6  10 ...  3  2  0  4  2  2  6  4  4  M
9    2  3  2  0  6  5  4  4  6  10 ...  4  1  0  4  5  4  6  4  9  M
10   2  2  2  1  6  2  2  4  3  10 ...  3  2  1  4  0  0  3  3  4  M
11   2  2  2  1  6  2  2  4  3  10 ...  2  2  1  4  2  1  3  3  4  M
12   3  3  3  2  6  5  4  4  6  10 ...  3  3  1  4  1  1  3  3  4  M
13   2  3  2  1  6  2  2  4  3  10

100.0

In [298]:
df=preprocess("wine.data",1,"none")
df= DataFrame(df)

classes= df[0]
for i in range(13):
    k=0
    k=df[i].astype(float).max()
    for j in range(178):
        df[i][j]= math.floor(math.ceil(float(df[i][j]) * 5) / k)
print (df)
df.to_csv(r'C:\Users\ANEESH CHATTARAJ\Desktop\comp30027\2020\assignment1_2020_datasets\Name111.data', index = False)
evaluate('Name111.data',14,"none")
    
    

    0  1  2  3  4  5  6  7  8  9  10 11 12 13
0    4  1  4  2  3  3  3  3  3  2  3  5  3  1
1    4  1  3  1  3  3  2  3  1  1  3  4  3  1
2    4  2  4  3  3  3  3  3  4  2  3  4  3  1
3    4  1  4  2  3  5  3  3  3  3  2  4  4  1
4    4  2  4  3  3  3  2  3  2  1  3  3  2  1
5    4  1  4  2  3  4  3  3  2  2  3  3  4  1
6    4  1  4  2  2  3  2  3  2  2  3  4  3  1
7    4  1  4  2  3  3  2  3  1  2  3  4  3  1
8    5  1  3  2  2  3  2  3  2  2  3  3  3  1
9    4  1  3  2  3  3  3  3  2  2  3  4  3  1
10   4  1  3  3  3  3  3  3  3  2  4  4  4  1
11   4  1  3  2  2  2  2  3  2  1  3  3  3  1
12   4  1  4  2  2  3  2  3  2  2  3  3  3  1
13   4  1  3  1  2  4  3  4  4  2  4  3  3  1
14   4  1  3  2  3  4  3  3  4  2  3  3  4  1
15   4  1  4  2  3  3  2  3  2  2  4  3  3  1
16   4  1  4  3  3  3  3  3  2  2  3  3  3  1
17   4  1  4  3  3  3  3  3  2  2  3  3  3  1
18   4  1  4  2  3  4  3  3  2  3  4  3  5  1
19   4  2  4  2  3  3  3  1  2  2  2  4  2  1
20   4  1  3  2  3  3  3  3  3  2 

96.64804469273743

### Q2
Implement a baseline model (e.g., random or 0R) and compare the performance of the na¨ıve Bayes classifier to this baseline on multiple datasets. Discuss why the baseline performance varies across datasets, and to what extent the na¨ıve Bayes classifier improves on the baseline performance.

In [327]:
def zero_rule_algorithm_classification(train, test):
    output_values = [row[-1] for row in train]
    prediction = max(set(output_values), key=output_values.count)
    predicted = [prediction for i in range(len(test))]
    return predicted
df=preprocess("wdbc.data",2,0)

total=0
k=zero_rule_algorithm_classification(df,df)
output_values = [row[-1] for row in df]

for a in range(len(df)):
      
        if  k[a]== output_values[a]:
            total=total+1
        accuracy= total/len(df)
print(accuracy)


0.6274165202108963


### Q3
Since it’s difficult to model the probabilities of ordinal data, ordinal attributes are often treated as either nominal variables or numeric variables. Compare these strategies on the ordinal datasets provided. Deterimine which approach gives higher classification accuracy and discuss why.

### Q4
Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy (you should implement this yourself and do not simply call existing implementations from `scikit-learn`). How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)

### Q5
Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the na¨ıve Bayes classifier? Explain why, or why not.

### Q6
The Gaussian na¨ıve Bayes classifier assumes that numeric attributes come from a Gaussian distribution. Is this assumption always true for the numeric attributes in these datasets? Identify some cases where the Gaussian assumption is violated and describe any evidence (or lack thereof) that this has some effect on the NB classifier’s predictions.